In [1]:
import sys
import os


project_root = os.path.abspath("..")  
sys.path.append(project_root)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../data/df_filtered.csv',index_col=False)
df['Unnamed: 0']
df = df.drop(columns='Unnamed: 0')

In [4]:
df['User_ID'].count()

418234

In [5]:
df["Watched"] = 1  # Раз есть в датасете — значит, юзер его посмотрел


In [6]:
df=df.drop(columns='Liked')

In [7]:
import numpy as np


df_negative = df.copy()


def add_negative_samples(user_data, all_animes):
    watched_anime = set(user_data["Anime_ID"].values)  # Смотренные аниме
    not_watched_anime = list(all_animes - watched_anime)  # Все аниме, которые не смотрел

    # Если у пользователя есть непросмотренные аниме, добавляем их
    if len(not_watched_anime) > 0:
        negative_anime = np.random.choice(not_watched_anime, size=len(user_data), replace=True)
        user_data["Anime_ID"] = negative_anime
        user_data["Watched"] = 0  # Устанавливаем "непросмотренное"
    
    return user_data


all_animes = set(df["Anime_ID"])


df_negative = df.groupby("User_ID").apply(lambda x: add_negative_samples(x, all_animes))

df_with_negatives = pd.concat([df, df_negative])


df_with_negatives = df_with_negatives.drop_duplicates(subset=["User_ID", "Anime_ID"])

del df_negative
del df
print(df_with_negatives["Watched"].value_counts())


/tmp/ipykernel_3925/1878543814.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_negative = df.groupby("User_ID").apply(lambda x: add_negative_samples(x, all_animes))


Watched
1    418234
0    409577
Name: count, dtype: int64


In [8]:
#N-1 сэпмплинг, так как у меня не будет новых данных
test = df_with_negatives.groupby("User_ID").sample(n=1, random_state=42)

train = df_with_negatives.drop(test.index)

print(f"Train size: {train.shape[0]}, Test size: {test.shape[0]}")

Train size: 823753, Test size: 4058


In [9]:
from catboost import CatBoostClassifier

features = ["User_ID", "Anime_ID", "type", "rating", "episodes", "members_log"] + list(df_with_negatives.columns[7:-2])  
target = "Watched"

X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

cat_features = ["User_ID", "Anime_ID", "type"]

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.01,
    depth=4,
    cat_features=cat_features,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=100
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)


0:	test: 0.9560290	best: 0.9560290 (0)	total: 425ms	remaining: 3m 32s
100:	test: 0.9908743	best: 0.9908743 (100)	total: 20.5s	remaining: 1m 21s
200:	test: 0.9939855	best: 0.9939855 (200)	total: 39.1s	remaining: 58.2s
300:	test: 0.9953234	best: 0.9953234 (300)	total: 1m 1s	remaining: 40.4s
400:	test: 0.9965854	best: 0.9965900 (399)	total: 1m 23s	remaining: 20.6s
499:	test: 0.9968818	best: 0.9968818 (499)	total: 1m 45s	remaining: 0us

bestTest = 0.9968818062
bestIteration = 499



In [10]:
model.save_model("../models/catboost_recommender.cbm")

In [11]:
def hit_rate_at_k(model, test, train, k=5):
    hits = 0
    total_users = test["User_ID"].nunique()

    for user in test["User_ID"].unique():
        # История просмотров пользователя
        watched_anime = set(train.loc[train["User_ID"] == user, "Anime_ID"].values)

        # Формируем список кандидатов (весь известный датасет, без фильтрации)
        candidates = test[test["User_ID"] == user][["User_ID", "Anime_ID"]].copy()
        # Предсказываем вероятность лайка
        preds = model.predict_proba(candidates.drop(columns=["Anime_ID"]))[:, 1]
        candidates["score"] = preds

        # Берем топ-K рекомендаций
        top_k = candidates.nlargest(k, "score")["Anime_ID"].values

        # Проверяем, есть ли среди рекомендаций хоть одно просмотренное
        if any(anime in watched_anime for anime in top_k):
            hits += 1

    return hits / total_users  # Усредняем по всем пользователям


In [12]:
hitrate = hit_rate_at_k(model, test, train, k=5)
print(f"Hitrate@5: {hitrate:.4f}")

CatBoostError: Invalid cat_features[1] = 1 value: index must be < 1.